### OICM Inference for models deployed with: vLLM, TGI ans SGLang
##### ref: https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html

In [1]:
import requests
import json
import os
from openai import OpenAI

In [2]:
api_key = os.getenv('OI_API_KEY') # Your valid api key
deployment_id = os.getenv('OI_DEPLOYMENT_ID') # deployment id
base_url = f"https://inference.develop.openinnovation.ai/models/{deployment_id}/proxy/v1" # change "develop" to your environment
model_name = "Qwen/Qwen2-2B-Instruct" # HF Model name ex: tiiuae/Falcon3-1B-Instruct

headers = {
    "Authorization": f"Bearer {api_key}"
}

### Using OpenAI API Client

In [4]:
client = OpenAI(
    base_url=base_url,
    api_key=api_key
)

In [7]:
stream = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You're a helpful assistant"},
                {"role": "user", "content": "what do you know about Egypt?"}
                
            ],
            max_tokens=256,
            temperature=0.7,
            stream=True
        )
        
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

I have a wealth of knowledge about Egypt, a country in northeastern Africa with a rich history, culture, and natural beauty. Here are some interesting facts about Egypt:

**History:**

1. Ancient Egypt: The civilization dates back to around 3100 BCE, with the unification of Upper and Lower Egypt by King Menes. The ancient Egyptians built the Great Pyramids of Giza, temples, tombs, and other monumental structures.
2. Roman and Islamic Periods: Egypt was a Roman province from 30 BCE to 641 CE, when it was conquered by the Arabs. In 1517, the Ottoman Empire took control, and in 1922, Egypt gained independence.
3. Modern Egypt: Since its independence, Egypt has experienced various governments, including a monarchy, a republic, and a military dictatorship.

**Landmarks and Attractions:**

1. Pyramids of Giza: The Great Pyramid of Khufu, the Pyramid of Khafre, and the Pyramid of Menkaure are the most famous pyramids in Egypt.
2. The Great Sphinx: A giant limestone statue with the body of a l

### Using requests

In [10]:
payload = {
    "messages": [
        {"role": "system", "content": "You're a helpful assistant"},
        {"role": "user", "content": "what do you know about Egypt?"}
    ],
    "max_tokens": 256,
    "temperature": 0.7,
    "model": model_name,
    "stream": True
}

response = requests.post(f"{base_url}/chat/completions", json=payload, headers=headers, stream=True)

for token in response.iter_lines():
    try:
        decoded_token = token.decode("UTF-8")
        json_str = decoded_token[6:]
        token_json = json.loads(json_str)
        new_str = token_json['choices'][0]['delta']['content']
        print(new_str, end="", flush=True)
    except:
        pass

I can provide you with a wealth of information about Egypt. Here are some of the key facts and attractions:

**Geography and Climate:**
Egypt is a transcontinental country, spanning across northeastern Africa and the Sinai Peninsula in the Middle East. It borders the Mediterranean Sea to the north, the Gaza Strip and Israel to the northeast, the Red Sea to the east, Sudan to the south, and Libya to the west. The country has a long, narrow shape, with the Nile River running through it.

**History:**
Egypt has a rich and diverse history, with evidence of human habitation dating back to around 6000 BCE. The ancient civilization of Egypt is famous for its pyramids, mummies, pharaohs, and temples. The Old Kingdom (2613-2181 BCE) saw the construction of the Great Pyramids at Giza, while the New Kingdom (1570-1085 BCE) was a period of significant cultural and economic growth.

**Culture:**
Egyptian culture is a blend of African, Arab, and Mediterranean influences. The country is home to many 

### Get the usage data
##### supported by vLLM and TGI
##### input_tokens and completion_tokens

##### Streaming mode

In [14]:
# add {"stream_options": {"include_usage": True}}

payload = {
    "messages": [
        {"role": "system", "content": "You're a helpful assistant"},
        {"role": "user", "content": "what do you know about Egypt?"}
    ],
    "max_tokens": 256,
    "temperature": 0.7,
    "model": model_name,
    "stream": True,
    "stream_options": {
        "include_usage": True
    }
}

# The usage is returned in the last token

response = requests.post(f"{base_url}/chat/completions", json=payload, headers=headers, stream=True)

for token in response.iter_lines():
    try:
        decoded_token = token.decode("UTF-8")
        json_str = decoded_token[6:]
        token_json = json.loads(json_str)
        choice = token_json['choices'][0]
        new_str = choice['delta']['content']
        print(new_str, end="", flush=True)
        if 'finish_reason' in choice and choice['finish_reason'] is not None:
            print("\n\n")
            print("=" * 100)
            print
            print(token_json)
            print(token_json["usage"])
            print("=" * 100)
    except:
        pass

Egypt is a country located in Northeast Africa, bordered by the Mediterranean Sea to the north, the Gaza Strip and Israel to the northeast, the Red Sea to the east, Sudan to the south, and Libya to the west. Here are some interesting facts about Egypt:

1. **Ancient History**: Egypt is home to some of the most ancient and influential civilizations in human history, including the Old Kingdom, the New Kingdom, and the Roman Period. The pyramids of Giza, built around 2580 BC, are an iconic symbol of Egypt's rich history and culture.

2. **Pyramids of Giza**: The Pyramids of Giza are not only a UNESCO World Heritage site but also one of the Seven Wonders of the Ancient World. The Great Pyramid of Giza, built for Pharaoh Khufu, is the largest pyramid and the largest man-made structure in the world.

3. **Nile River**: The Nile River is the longest river in the world and the primary source of water and fertility for Egypt. It flows through the heart of Egypt and has supported the country's a

##### non-streaming mode

In [17]:
# {"stream_options": {"include_usage": True}} is not permited as the request is not streaming

payload = {
    "messages": [
        {"role": "system", "content": "You're a helpful assistant"},
        {"role": "user", "content": "what do you know about Egypt?"}
    ],
    "max_tokens": 256,
    "temperature": 0.7,
    "model": model_name,
}

# The usage is returned in the last token

response = requests.post(f"{base_url}/chat/completions", json=payload, headers=headers)

usage = response.json().get("usage")
usage

{'prompt_tokens': 47, 'total_tokens': 303, 'completion_tokens': 256}

### Integrate with Langchain

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
        model_name=model_name,
        openai_api_key=api_key,
        openai_api_base=base_url,
        max_tokens=1000,
)

llm_answer = llm.invoke("Egypt is")

llm_answer.content

"Egypt is a country located in Northeast Africa, bordered by Sudan to the north, South Sudan to the east, Ethiopia to the northwest, and Libya to the west. It is also connected to the Mediterranean Sea to the south through the Suez Canal. The capital of Egypt is Cairo, which is also the largest city in the country. The modern nation-state of Egypt was formed in 1922 following the Arab nationalist struggle for independence from British colonial rule. Egypt has a rich history spanning over 5,000 years, making it one of the world's oldest civilizations."

### Langchain Structured Output
#### Structured output is only supported with models deployed with vLLM

In [44]:
from typing import List, Optional
from pydantic import BaseModel, Field

In [46]:
class BookRecommendation(BaseModel):
    """Details of a book recommendation."""
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")
    genre: str = Field(description="The genre of the book")
    rating: float = Field(description="The book's rating on a scale from 1.0 to 5.0")
    reasons: List[str] = Field(
        description="Reasons why this book is recommended"
    )

In [50]:
structured_llm = llm.with_structured_output(BookRecommendation)

response = structured_llm.invoke("Recommend a good science fiction book for someone who loves space exploration.")

response

BookRecommendation(title='The Expanse', author='James S. A. Corey', genre='Science Fiction', rating=8.0, reasons=['The Expanse is a critically acclaimed series that explores a future where humanity has colonized the solar system and faces challenges such as resource scarcity, political conflict, and alien threats.', 'It delves into the complexities of human nature, ethics, and survival in a universe where boundaries blur.', 'Its rich world-building and intricate plotlines make it a compelling read for fans of space exploration and speculative fiction.', 'The series offers a deep dive into the consequences of human actions in a universe governed by gravity, technology, and the stars.', 'Readers will find the blend of suspense, character development, and political intrigue engaging and thought-provoking.', "Overall, 'The Expanse' is a solid choice for those who enjoy the depth and complexity of science fiction narratives set in the vast expanse of the cosmos."])